In [1]:
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

physical_devices = tf.config.list_physical_devices('GPU')
print('Physical Devices', physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Physical Devices [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Load data

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
x_train = x_train.reshape(-1, 28*28).astype('float32') /  255.
x_test = x_test.reshape(-1, 28*28).astype('float32') /  255.

In [5]:
type(x_train)

numpy.ndarray

In [6]:
x_train = tf.convert_to_tensor(x_train)
x_test = tf.convert_to_tensor(x_test)

### Model with Sequential API

In [7]:
model = keras.Sequential([
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(10)
])

In [8]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [9]:
model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)

Epoch 1/5
1875/1875 - 5s - loss: 0.1852 - accuracy: 0.9432 - 5s/epoch - 3ms/step
Epoch 2/5
1875/1875 - 4s - loss: 0.0787 - accuracy: 0.9752 - 4s/epoch - 2ms/step
Epoch 3/5
1875/1875 - 4s - loss: 0.0548 - accuracy: 0.9829 - 4s/epoch - 2ms/step
Epoch 4/5
1875/1875 - 4s - loss: 0.0408 - accuracy: 0.9867 - 4s/epoch - 2ms/step
Epoch 5/5
1875/1875 - 4s - loss: 0.0314 - accuracy: 0.9902 - 4s/epoch - 2ms/step


#### Getting model summary while using Sequential API

In [10]:
model = keras.Sequential([
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(10)
])

model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [11]:
model = keras.Sequential([
    keras.Input(shape=(28*28)),
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(10)
])

In [12]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 512)               401920    
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dense_8 (Dense)             (None, 10)                2570      
                                                                 
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________


#### Building model using add() method 

In [13]:
model = keras.Sequential()

In [14]:
model.add(keras.Input(shape=(28*28)))
model.add(layers.Dense(512, activation='relu'))
print('==================')
print(model.summary())
print('==================')
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
print('==================')
print(model.summary())
print('==================')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 512)               401920    
                                                                 
Total params: 401,920
Trainable params: 401,920
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 512)               401920    
                                                                 
 dense_10 (Dense)            (None, 256)               131328    
                                                                 
 dense_11 (Dense)            (None, 10)                2570      
                                                                 
Total params: 535,818

### Building models using the Functional API

In [15]:
inputs = keras.Input(shape=(28*28))
x = layers.Dense(512, activation='relu', name='first_layer')(inputs)
x = layers.Dense(256, activation='relu', name='second_layer')(x)
outputs = layers.Dense(512, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [16]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [17]:
model.fit(x_train, y_train, batch_size=32, epochs=5, verbose=2)

Epoch 1/5
1875/1875 - 4s - loss: 0.2239 - accuracy: 0.9368 - 4s/epoch - 2ms/step
Epoch 2/5
1875/1875 - 4s - loss: 0.0833 - accuracy: 0.9736 - 4s/epoch - 2ms/step
Epoch 3/5
1875/1875 - 4s - loss: 0.0554 - accuracy: 0.9826 - 4s/epoch - 2ms/step
Epoch 4/5
1875/1875 - 4s - loss: 0.0403 - accuracy: 0.9871 - 4s/epoch - 2ms/step
Epoch 5/5
1875/1875 - 4s - loss: 0.0337 - accuracy: 0.9887 - 4s/epoch - 2ms/step


### Pulling output from specific layers

#### Get started with pulling the output of the penultimate layer

In [18]:
model = keras.Sequential()

In [19]:
model.add(keras.Input(shape=(28*28)))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(10, activation='relu'))

In [20]:
model = keras.Model(inputs = model.inputs, outputs = [model.layers[-2].output])

In [21]:
feature = model.predict(x_train)

1875/1875 [==============================] - 2s 1ms/step


In [22]:
print(feature.shape)

(60000, 256)


#### Naming layers and getting output from those layers by using their name using [model].get_layer()

In [23]:
model = keras.Sequential()
model.add(keras.Input(shape=(28*28)))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu', name='my_layer'))
model.add(layers.Dense(10, activation='relu'))

In [25]:
model.get_layer('my_layer')

In [26]:
model = keras.Model(inputs = model.inputs, outputs = [model.get_layer('my_layer').output])

In [27]:
feature = model.predict(x_train)

1875/1875 [==============================] - 2s 1ms/step


In [28]:
print(feature.shape)

(60000, 256)


#### Getting output from all layers

In [29]:
model = keras.Sequential()
model.add(keras.Input(shape=(28*28)))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu', name='my_layer'))
model.add(layers.Dense(10, activation='relu'))

In [30]:
model.layers

In [31]:
model = keras.Model(inputs = model.inputs, outputs = [layer.output for layer in model.layers])

In [32]:
outputs = model.predict(x_train)

1875/1875 [==============================] - 3s 1ms/step


In [33]:
for output in outputs:
    print(output.shape)

(60000, 512)
(60000, 256)
(60000, 10)
